In [1]:
import json
import codecs
import re
import pandas as pd

In [2]:
import nltk
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
import nltk
from num2words import num2words
from nltk.stem.snowball import PortugueseStemmer
import string

In [3]:
from sklearn.utils import shuffle

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression  # Logistic Regression
from sklearn.cross_validation import train_test_split # Para dividir o conjunto de treinamento e teste
from sklearn.neighbors import KNeighborsClassifier  # K nearest neighbours
from sklearn import svm  # Para o algoritmo Support Vector Machine (SVM) Algorithm
from sklearn import metrics # Para verificar as métricas
from sklearn.tree import DecisionTreeClassifier # para o algoritmo de árvores de decisão
from sklearn.neural_network import MLPClassifier # Para as redes neurais
from sklearn import ensemble, naive_bayes, neighbors, svm, tree
from sklearn.preprocessing import MinMaxScaler

c:\users\allanbs\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
c:\users\allanbs\appdata\local\programs\python\python36\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
def beep():
    import winsound
    duration = 1000  # millisecond
    freq = 440  # Hz
    winsound.Beep(freq, duration)

Metodo para ler o arquivo json recebido

In [5]:
def read_json_file(path, enc='utf8'):
    with codecs.open(path, encoding=enc) as j:
        data_json = json.load(j)
    return data_json

Metodo para remover o index do documento (cabecalho)

In [6]:
def remove_document_index(x):
    return re.sub('.*(?<=\\r1)\.', '', x)

Método para filtrar as stopwords

In [7]:
def filter_stopwords(tokens):
    """Docstring."""
    return [i.lower() for i in tokens if
            i.lower() not in stopwords] 

Método para aplicar o stemming - Note que neste caso a pontuacao foi removida

In [8]:
def stemming(x):
    return _stemmer.stem(x)

Transforma os digitos em texto (e.g. 1 -> um, 2 -> dois)

In [9]:
def number_to_word(word, language = 'pt_BR'):
    try:
        return num2words(float(word), to = 'cardinal', lang = language)
    except NotImplementedError:
        return word

Extrai as metricas do modelo

In [43]:
def extract_model_metrics(y_predicted, y_test, positive_label=1):
        model_metrics = dict()
        model_metrics['accuracy'] = metrics.accuracy_score(y_test, y_predicted)
        model_metrics['f1'] = metrics.f1_score(y_test, y_predicted, average='weighted', pos_label=1)
        model_metrics['precision'] = metrics.precision_score(y_test, y_predicted, average='weighted', pos_label=1)
        model_metrics['recall'] = metrics.recall_score(y_test, y_predicted, average='weighted', pos_label=1)

        fpr, tpr, threshold = metrics.roc_curve(
            y_test, 
            y_predicted.tolist()
        )

        # TODO: Rever isso, pois é sobreescrito em seguida
        model_metrics['true_positive'] = tpr
        model_metrics['false_positive'] = fpr
        model_metrics['auc'] = metrics.auc(fpr, tpr)
        model_metrics['kappa'] = metrics.cohen_kappa_score(y_test, y_predicted)
        model_metrics['log_loss'] = metrics.log_loss(y_test, y_predicted)

        confusion_matrix = pd.DataFrame(metrics.confusion_matrix(y_test, y_predicted))

        _tp = confusion_matrix.iloc[1,1]
        _tn = confusion_matrix.iloc[0,0]
        _fp = confusion_matrix.iloc[0,1]
        _fn = confusion_matrix.iloc[1,0]

        model_metrics['true_positive'] = confusion_matrix.iloc[1,1]
        model_metrics['true_negative'] = confusion_matrix.iloc[0,0]
        model_metrics['false_positive'] = confusion_matrix.iloc[0,1]
        model_metrics['false_negative'] = confusion_matrix.iloc[1,0]

        model_metrics['positive_pred_value'] = 0.0 if (_tp + _fp) == 0 else (_tp / (_tp + _fp))
        model_metrics['negative_pred_value'] = 0.0 if (_tn + _fn) == 0 else (_tn / (_tn + _fn))

        model_metrics['sensitivity'] = 0.0 if (_tp + _fn) == 0 else (_tp / (_tp + _fn))
        model_metrics['specificity'] = 0.0 if (_tn + _fp) == 0 else (_tn / (_tn + _fp))

        model_metrics['expected_no'] = Counter(y_test)[0]
        model_metrics['expected_yes'] = Counter(y_test)[1]
        model_metrics['diff_expected'] = abs(Counter(y_test)[1] - Counter(y_predicted)[1])

        for x in model_metrics.keys():
            model_metrics[x] = float(model_metrics[x])
        
        return model_metrics

Definições globais

In [11]:
_stemmer = PortugueseStemmer()
stopwords = nltk.corpus.stopwords.words('portuguese')
punct = string.punctuation

In [12]:
data_civil = read_json_file('C:/Users/allanbs/Documents/Git/RicardoDissertacao/dados/Formato Json/Civel.json')

In [13]:
data_crime = read_json_file('C:/Users/allanbs/Documents/Git/RicardoDissertacao/dados/Formato Json/Crime.json')

In [14]:
data_crime = data_crime['Documentos']
k = list()
for x in data_crime:
    k.append(remove_document_index(x['EMENTA']))

data_crime = k

In [15]:
data_civil = data_civil['Documentos']
k = list()
for x in data_civil:
    k.append(remove_document_index(x['EMENTA']))

data_civil = k

Remove o cabeçalho e caracteres de controle

In [16]:
for i in range(0, len(data_civil)):
    x = data_civil[i]
    x = re.sub('.*(?<=\\r1)\.', '', x)
    x = re.sub(r'[\t\n\r]', ' ', x)
    x = re.sub("\s\s+", " ", x)
    data_civil[i] = x

In [17]:
for i in range(0, len(data_crime)):
    x = data_crime[i]
    x = re.sub('.*(?<=\\r1)\.', '', x)
    x = re.sub(r'[\t\n\r]', ' ', x)
    x = re.sub("\s\s+", " ", x)
    data_crime[i] = x

Aplica o filtro por stopwords em todos os textos

In [18]:
data_civil = [' '.join(filter_stopwords(x.split(' '))) for x in data_civil]
data_crime = [' '.join(filter_stopwords(x.split(' '))) for x in data_crime]

Aplica o filtro o stemming em todos os textos

In [19]:
data_civil = [stemming(x) for x in data_civil]
data_crime = [stemming(x) for x in data_crime]

Cria o vectorizer para processar todos os textos de TF-IDF

In [20]:
vectorizer = TfidfVectorizer()

Repassa todos os textos para o TF-IDF

In [21]:
texts = data_civil
texts.extend(data_crime)
vectorizer.fit(texts)
del texts

Obtém o vetor TF-IDF para o primeiro elemento de todos os textos

In [22]:
data_civil = vectorizer.transform(data_civil).toarray()
data_crime = vectorizer.transform(data_crime).toarray()

Gera os dataframes e atribui os labels dos dados

In [23]:
data_civil = pd.DataFrame(data_civil,columns=vectorizer.get_feature_names())
data_crime = pd.DataFrame(data_crime,columns=vectorizer.get_feature_names())

In [24]:
data_civil['CLASSE'] = 'civil'
data_crime['CLASSE'] = 'crime'

In [25]:
all_data = pd.concat([data_civil, data_crime], axis=0)

In [26]:
all_data = shuffle(all_data)

In [27]:
beep()

Divide o conjunto em treinamento (70%) e teste (30%)

In [44]:
all_data['CLASSE'] = [1 if x == 'crime' else 0 for x in all_data['CLASSE']]
x_train, x_test, y_train, y_test = train_test_split(all_data.drop('CLASSE', axis=1, inplace=False), all_data['CLASSE'], test_size = 0.3, random_state = 42)

Configuração da semente

In [30]:
random_state_seed = 2567

Aplica o SVM com kernel linear

In [ ]:
model = svm.SVC(kernel='linear', gamma='auto', C=1, degree=0.1, probability=False, random_state=random_state_seed)
model.fit(x_train,y_train) # nós treinamos o algoritmo com os dados de treinamento e a saída de treinamento
y_predicted=model.predict(x_test) # agora passamos os dados de teste para o algoritmo treinado

# Para verificar o desempenho, é necessário passar a saída obtida pelo modelo e a esperada
extract_model_metrics(y_predicted,y_test) # agora nós verificamos a acurácia do algoritmo.


{'accuracy': 0.4898148148148148,
 'auc': 0.4201102034685874,
 'diff_expected': 83.0,
 'expected_no': 3601.0,
 'expected_yes': 1799.0,
 'f1': 0.48669763105590147,
 'false_negative': 1419.0,
 'false_positive': 1336.0,
 'kappa': -0.16164559666278122,
 'log_loss': 17.621369857434384,
 'negative_pred_value': 0.6148208469055375,
 'positive_pred_value': 0.22144522144522144,
 'precision': 0.4837684857568137,
 'recall': 0.4898148148148148,
 'sensitivity': 0.2112284602556976,
 'specificity': 0.6289919466814774,
 'true_negative': 2265.0,
 'true_positive': 380.0}

Aplica o SVM com kernel radial

In [60]:
model = svm.SVC(kernel='rbf', gamma='auto', C=1, degree=0.1, probability=False, random_state=random_state_seed)
model.fit(x_train,y_train) # nós treinamos o algoritmo com os dados de treinamento e a saída de treinamento
y_predicted=model.predict(x_test) # agora passamos os dados de teste para o algoritmo treinado

# Para verificar o desempenho, é necessário passar a saída obtida pelo modelo e a esperada
extract_model_metrics(y_predicted,y_test) # agora nós verificamos a acurácia do algoritmo.


c:\users\allanbs\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\allanbs\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


{'accuracy': 0.6668518518518518,
 'auc': 0.5,
 'diff_expected': 1799.0,
 'expected_no': 3601.0,
 'expected_yes': 1799.0,
 'f1': 0.5335703851835393,
 'false_negative': 1799.0,
 'false_positive': 0.0,
 'kappa': 0.0,
 'log_loss': 11.506529395267465,
 'negative_pred_value': 0.6668518518518518,
 'positive_pred_value': 0.0,
 'precision': 0.4446913923182441,
 'recall': 0.6668518518518518,
 'sensitivity': 0.0,
 'specificity': 1.0,
 'true_negative': 3601.0,
 'true_positive': 0.0}

Aplica o SVM com kernel polinomial

In [ ]:
model = svm.SVC(kernel='poly', gamma='auto', C=1, degree=0.1, probability=False, random_state=random_state_seed)
model.fit(x_train,y_train) # nós treinamos o algoritmo com os dados de treinamento e a saída de treinamento
y_predicted=model.predict(x_test) # agora passamos os dados de teste para o algoritmo treinado

# Para verificar o desempenho, é necessário passar a saída obtida pelo modelo e a esperada
extract_model_metrics(y_predicted,y_test) # agora nós verificamos a acurácia do algoritmo.


Aplica a regressão logistica

In [53]:
model = LogisticRegression(random_state=random_state_seed)
model.fit(x_train,y_train)
y_predicted=model.predict(x_test)
extract_model_metrics(y_predicted,y_test)

{'accuracy': 0.5185185185185185,
 'auc': 0.44218833042949124,
 'diff_expected': 230.0,
 'expected_no': 3601.0,
 'expected_yes': 1799.0,
 'f1': 0.5095286499179021,
 'false_negative': 1415.0,
 'false_positive': 1185.0,
 'kappa': -0.11944421986458553,
 'log_loss': 16.629956694393066,
 'negative_pred_value': 0.6306447402766902,
 'positive_pred_value': 0.2447418738049713,
 'precision': 0.502081914946575,
 'recall': 0.5185185185185185,
 'sensitivity': 0.21345191773207337,
 'specificity': 0.6709247431269092,
 'true_negative': 2416.0,
 'true_positive': 384.0}

Aplica árvores de decisão

In [58]:
model=DecisionTreeClassifier(random_state=random_state_seed)
model.fit(x_train,y_train)
y_predicted=model.predict(x_test)
extract_model_metrics(y_predicted,y_test)

{'accuracy': 0.43203703703703705,
 'auc': 0.33979312768872955,
 'diff_expected': 303.0,
 'expected_no': 3601.0,
 'expected_yes': 1799.0,
 'f1': 0.4173880151013432,
 'false_negative': 1685.0,
 'false_positive': 1382.0,
 'kappa': -0.3345106262558024,
 'log_loss': 19.616950416081405,
 'negative_pred_value': 0.5683913934426229,
 'positive_pred_value': 0.07620320855614973,
 'precision': 0.40441981110729597,
 'recall': 0.43203703703703705,
 'sensitivity': 0.06336853807670928,
 'specificity': 0.6162177173007498,
 'true_negative': 2219.0,
 'true_positive': 114.0}

Aplica o algoritmo do KNN

In [59]:
model=KNeighborsClassifier(n_neighbors=3)
model.fit(x_train,y_train)
y_predicted=model.predict (x_test)
extract_model_metrics(y_predicted,y_test)

{'accuracy': 0.4875925925925926,
 'auc': 0.42136471880533466,
 'diff_expected': 29.0,
 'expected_no': 3601.0,
 'expected_yes': 1799.0,
 'f1': 0.4865383104948638,
 'false_negative': 1398.0,
 'false_positive': 1369.0,
 'kappa': -0.15790746225629904,
 'log_loss': 17.69812758029626,
 'negative_pred_value': 0.6148760330578512,
 'positive_pred_value': 0.22655367231638418,
 'precision': 0.48550715769231434,
 'recall': 0.4875925925925926,
 'sensitivity': 0.22290161200667039,
 'specificity': 0.6198278256039988,
 'true_negative': 2232.0,
 'true_positive': 401.0}

Aplica o algoritmo Random Forest

In [61]:
model=ensemble.RandomForestClassifier(n_estimators=100, criterion='entropy', max_depth=None, n_jobs=-1, bootstrap=True, random_state=random_state_seed)
model.fit(x_train,y_train)
y_predicted=model.predict(x_test)
extract_model_metrics(y_predicted,y_test)

{'accuracy': 0.4324074074074074,
 'auc': 0.3449386936091342,
 'diff_expected': 235.0,
 'expected_no': 3601.0,
 'expected_yes': 1799.0,
 'f1': 0.42154387757039086,
 'false_negative': 1650.0,
 'false_positive': 1415.0,
 'kappa': -0.32060121944369246,
 'log_loss': 19.604163163104623,
 'negative_pred_value': 0.5698644421272159,
 'positive_pred_value': 0.09526854219948849,
 'precision': 0.4117536969475896,
 'recall': 0.4324074074074074,
 'sensitivity': 0.08282379099499722,
 'specificity': 0.6070535962232713,
 'true_negative': 2186.0,
 'true_positive': 149.0}

Aplica o algoritmo Naive Bayes

In [55]:
model=naive_bayes.GaussianNB()
model.fit(x_train,y_train)
y_predicted=model.predict(x_test)
extract_model_metrics(y_predicted,y_test)

{'accuracy': 0.5031481481481481,
 'auc': 0.4739182139974397,
 'diff_expected': 475.0,
 'expected_no': 3601.0,
 'expected_yes': 1799.0,
 'f1': 0.5145783949444179,
 'false_negative': 1104.0,
 'false_positive': 1579.0,
 'kappa': -0.04893100000086892,
 'log_loss': 17.160888820719933,
 'negative_pred_value': 0.6468330134357005,
 'positive_pred_value': 0.30562884784520666,
 'precision': 0.5331614775287934,
 'recall': 0.5031481481481481,
 'sensitivity': 0.38632573652028906,
 'specificity': 0.5615106914745904,
 'true_negative': 2022.0,
 'true_positive': 695.0}